In [23]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
import time
import concurrent.futures

def create_folder(f_name): 
    try:
        if not os.path.exists(f_name):
            os.makedirs(f_name)
    except OSError:
        print("The folder could not be created!")

def process_chunk(chunk):
    chunk = chunk.fillna(0)
    chunk["Label"] = chunk["Label"].apply(lambda x: 1 if x == "BENIGN" else 0)

    y = chunk["Label"].values
    chunk.drop(columns=["Label"], inplace=True)
    X = chunk.values.astype(np.float32)
    X[np.isnan(X)] = 0
    X[np.isinf(X)] = 0

    forest = RandomForestRegressor(n_estimators=250, random_state=0, n_jobs=-1)
    forest.fit(X, y)
    importances = forest.feature_importances_
    return importances

def process_csv_file(csv_file, main_labels, chunk_size=10000):
    importances_list = []

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for chunk in pd.read_csv(csv_file, usecols=main_labels, chunksize=chunk_size):
            futures.append(executor.submit(process_chunk, chunk))

        for future in concurrent.futures.as_completed(futures):
            importances_list.append(future.result())

    average_importances = np.mean(importances_list, axis=0)
    return average_importances

def save_importance_plot(importances, columns, csv_file):
    indices = np.argsort(importances)[::-1]
    top_features = [columns[i] for i in indices[:20]]
    top_importances = importances[indices[:20]]

    impor_bars = pd.DataFrame({'Features': top_features, 'Importance': top_importances}).set_index('Features')
    plt.rcParams['figure.figsize'] = (10, 5)
    impor_bars.plot.bar()

    plt.title(f"{csv_file[:-4]} Attack - Feature Importance")
    plt.ylabel('Importance')

    output_path = f"./feaure_pics/{csv_file[:-4]}.pdf"
    plt.savefig(output_path, bbox_inches='tight', format='pdf')
    plt.close()

    print(f"Plot saved to {output_path}")

    fea_ture = f"{csv_file[:-4]}=[" + ",".join([f"\"{feat}\"" for feat in top_features[:5]]) + "]"
    result = f"{csv_file[:-4]} importance list:\n{impor_bars.head(20)}\n\n\n{fea_ture}"
    return result

def main():
    start_time = time.time()

    # CSV file names
    csv_files = ["all_data.csv"]

    # Headers of columns
    main_labels = ["Flow Duration", "Total Fwd Packets", "Total Backward Packets", "Total Length of Fwd Packets", "Total Length of Bwd Packets", "Fwd Packet Length Max", "Fwd Packet Length Min",
                   "Fwd Packet Length Mean", "Fwd Packet Length Std", "Bwd Packet Length Max", "Bwd Packet Length Min", "Bwd Packet Length Mean", "Bwd Packet Length Std",
                   "Flow Bytes/s", "Flow Packets/s", "Flow IAT Mean", "Flow IAT Std", "Flow IAT Max", "Flow IAT Min", "Fwd IAT Total", "Fwd IAT Mean", "Fwd IAT Std", "Fwd IAT Max",
                   "Fwd IAT Min", "Bwd IAT Total", "Bwd IAT Mean", "Bwd IAT Std", "Bwd IAT Max", "Bwd IAT Min", "Fwd PSH Flags", "Bwd PSH Flags", "Fwd URG Flags", "Bwd URG Flags",
                   "Fwd Header Length", "Bwd Header Length", "Fwd Packets/s", "Bwd Packets/s", "Min Packet Length", "Max Packet Length", "Packet Length Mean", "Packet Length Std",
                   "Packet Length Variance", "FIN Flag Count", "SYN Flag Count", "RST Flag Count", "PSH Flag Count", "ACK Flag Count", "URG Flag Count", "CWE Flag Count",
                   "ECE Flag Count", "Down/Up Ratio", "Average Packet Size", "Avg Fwd Segment Size", "Avg Bwd Segment Size", "Fwd Avg Bytes/Bulk",
                   "Fwd Avg Packets/Bulk", "Fwd Avg Bulk Rate", "Bwd Avg Bytes/Bulk", "Bwd Avg Packets/Bulk", "Bwd Avg Bulk Rate", "Subflow Fwd Packets", "Subflow Fwd Bytes",
                   "Subflow Bwd Packets", "Subflow Bwd Bytes", "Init_Win_bytes_forward", "Init_Win_bytes_backward", "act_data_pkt_fwd",
                   "min_seg_size_forward", "Active Mean", "Active Std", "Active Max", "Active Min",
                   "Idle Mean", "Idle Std", "Idle Max", "Idle Min", "Label"]

    create_folder("./feaure_pics/")
    
    all_results = []
    for csv_file in csv_files:
        importances = process_csv_file(csv_file, main_labels)
        result = save_importance_plot(importances, main_labels[:-1], csv_file)
        all_results.append(result)

    with open("importance_list_all_data.csv", "w") as ths:
        for result in all_results:
            ths.write(result)
            print(result)
            print("-----------------------------------------------------------------------------------------------\n\n\n\n")

    print("Mission accomplished!")
    print("Total operation time: =", time.time() - start_time, "seconds")

if __name__ == "__main__":
    main()


Plot saved to ./feaure_pics/all_data.pdf
all_data importance list:
                         Importance
Features                           
Init_Win_bytes_forward     0.061494
Init_Win_bytes_backward    0.044521
Fwd Packet Length Max      0.041572
Bwd Packet Length Std      0.032616
Bwd Header Length          0.025558
Min Packet Length          0.017830
Fwd Packet Length Std      0.017545
Average Packet Size        0.016500
Fwd IAT Min                0.015423
Flow IAT Mean              0.015067
Flow IAT Max               0.013958
Flow IAT Min               0.013127
Packet Length Mean         0.012820
Flow Duration              0.012051
Bwd Packets/s              0.011286
act_data_pkt_fwd           0.010686
Bwd IAT Max                0.010555
Fwd Packet Length Mean     0.010355
Subflow Fwd Bytes          0.010341
Avg Fwd Segment Size       0.010255


all_data=["Init_Win_bytes_forward","Init_Win_bytes_backward","Fwd Packet Length Max","Bwd Packet Length Std","Bwd Header Length"]
---------